## **🔒문제설명**
---
* **제시된 이커머스 데이터를 대상으로 아래 조건을 포함하여 코호트 분석을 수행하시오**


> **전처리 기준**

```
(1) 'CustomerID'가 없는 row는 제거
(2) 중복되는 row는 제거 (※ row의 모든 column이 중복되는 데이터 제거)
(3) 'Quantity' 및 'UnitPrice'가 0 초과인 row만 분석
(4) 코호트(Cohort) 그룹은 'CustomerID' 기준으로 첫 구매한 달의 1일로 정의
```  


> **예상 Output**

```
(1) 코호트(Cohort) 그룹 및 분석기간은 2010-12월부터 ~ 2011~12월까지 분석
(2) Column명은 아래 제시된 'CohortIndex'와 'CohortMonth'를 사용
(3) CohortIndex : 유입할 달(Month)로 부터 지난 개월 수 (※0 이 아닌 1부터 시작)
(4) 코호트(Cohort) 그룹별 Unique한 고객 수를 카운팅하고, 최종 결과물은 %로 표현
(5) %는 소수 첫째 자리까지 출력, 둘째 자리에서 반올림 ex) 33.4%
(6) CohortIndex에 따른 유입 고객이 없는 코호트(Cohort) 그룹의 경우 NaN, Null 값 그대로 표현
```  



|CohortIndex|1|2|3|4|5|6|7|8|9|10|11|12|13|
|:---:|:---:|:---:|:---:|:---:|:---:|:---:|:---:|:---:|:---:|:---:|:---:|:---:|:---:|
|CohortMonth||||||||||||||
|2010-12-01|100.0|50.2|46.2|55.4|36.2|32.1|23.1|22.1|25.1|24.1|22.6|22.7|22.1|
|2011-01-01|100.0|34.5|22.1|25.4|33.2|21.3|22.1|22.4|20.1|18.0|19.0|15.2|NaN|
|...|...|...|...|...|...|...|...|...|...|...|...|...|...|

*※ 상위 숫자는 임의의 숫자, Column명 및 출력 값 형태는 동일

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [4]:
import pandas as pd
df = pd.read_csv('eda1_df.csv', encoding='ISO-8859-1')

In [5]:
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,17850.0,United Kingdom


In [23]:
# CustomerID가 없는 row 제거.
#df_clean = df['CustomerID'].dropna() -> 시리즈 형태로 반환하므로 subset으로 해당 열기준으로 drop해줘야한다.
df_clean = df.dropna(subset = ['CustomerID'], how= "all") #열 하나만 인덱싱해서 뽑으면 시리즈로 반환하니까 subset 안먹힘.
df_clean.isna().sum()

InvoiceNo      0
StockCode      0
Description    0
Quantity       0
InvoiceDate    0
UnitPrice      0
CustomerID     0
Country        0
dtype: int64

In [24]:
#중복데이터 확인. duplicated
df_clean.duplicated().sum()

np.int64(5225)

In [26]:
#중복데이터 날리기. drop_duplicates
df_clean = df_clean.drop_duplicates()

In [34]:
#조건 필터링
df_clean = df_clean[(df_clean['Quantity']>0) & (df_clean['UnitPrice']>0)]
round(df_clean.describe())

,Quantity,UnitPrice,CustomerID
count,524878.0,524878.0,392692.0
mean,11.0,4.0,15288.0
std,156.0,36.0,1714.0
min,1.0,0.0,12346.0
25%,1.0,1.0,13955.0
50%,4.0,2.0,15150.0
75%,11.0,4.0,16791.0
max,80995.0,13541.0,18287.0


In [36]:
#코호트 month 구하기
from datetime import datetime as dt
df_clean['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'])

def get_month(x) :
    return dt(x.year,x.month,1)

df_clean['InvoiceMonth'] = df_clean['InvoiceDate'].apply(get_month)
grouping = df_clean.groupby('CustomerID')['InvoiceMonth']

df_clean['CohortMonth'] = grouping.transform('min')
df_clean.tail()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,InvoiceMonth,CohortMonth
541904,581587,22613,PACK OF 20 SPACEBOY NAPKINS,12,2011-12-09 12:50:00,0.85,12680.0,France,2011-12-01,2011-08-01
541905,581587,22899,CHILDREN'S APRON DOLLY GIRL,6,2011-12-09 12:50:00,2.10,12680.0,France,2011-12-01,2011-08-01
541906,581587,23254,CHILDRENS CUTLERY DOLLY GIRL,4,2011-12-09 12:50:00,4.15,12680.0,France,2011-12-01,2011-08-01
541907,581587,23255,CHILDRENS CUTLERY CIRCUS PARADE,4,2011-12-09 12:50:00,4.15,12680.0,France,2011-12-01,2011-08-01
541908,581587,22138,BAKING SET 9 PIECE RETROSPOT,3,2011-12-09 12:50:00,4.95,12680.0,France,2011-12-01,2011-08-01
